In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRe

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-12-14 01:43:19--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  2.95MB/s    in 0.3s    

2020-12-14 01:43:19 (2.95 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_df=df.select(["customer_id","review_id","review_headline","review_body","review_date"])
review_df.show()

+-----------+--------------+--------------------+--------------------+-----------+
|customer_id|     review_id|     review_headline|         review_body|review_date|
+-----------+--------------+--------------------+--------------------+-----------+
|   32158956|R1KKOXHNI8MSXU|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|    2714559|R26SP2OPDK4HT7|Favorite for wint...|I love this dress...| 2014-03-04|
|   12608825| RWQEDYAX373I1|Great Socks for t...|Nice socks, great...| 2015-07-12|
|   25482800|R231YI7R4GPF6J|          Slick hat!|I bought this for...| 2015-06-03|
|    9310286|R3KO3W45DD0L1K|I would do it again!|Perfect dress and...| 2015-06-12|
|   26631939|R1C4QH63NFL5NJ|          Five Stars|Excellent for my ...| 2014-04-22|
|   48785098|R2GP65O1U9N7BP|            Love it!|Raw is the only w...| 2015-07-28|
|   39548589|R3O29CT5MQQ3XQ|         Three Stars|        A bit large.| 2015-07-10|
|   29355866|R1ZECD2AA8QFF6|          Five Stars|          Great fit!| 2015-08-09|
|   

In [ ]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").count().withColumnRenamed("count", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   13780696|             1|
|   48489393|             8|
|   43380682|             1|
|   17133428|             1|
|    9731896|             2|
|    1797768|             1|
|   27446622|             1|
|     120706|             3|
|   15915947|             2|
|   37520920|             1|
|   43920023|             9|
|   14309190|             8|
|   22316505|             1|
|   43949054|             2|
|   21102747|             1|
|   19021463|             1|
|   16075459|             8|
|   51475782|             2|
|   11555870|             5|
|   35471383|             3|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B01427DDXM|Zoozie LA Women's...|
|B013TPITFA|Tabeez Women's Pl...|
|B013FU4Y9E|Ekouaer Women's U...|
|B013FNK6PC|Ezcosplay Women O...|
|B013FACXVK|Womens Two Piece ...|
|B013DHKQ3W|OURS Women's 3 4 ...|
|B0131D4NKU|Genie Women's Jeg...|
|B0131CA9K4|BeWild Brand - Ju...|
|B0130ZDFTE|Flora&Fred Women'...|
|B012Y665WY|AW Fashion's I Ne...|
|B012X0G1KW|KAMACO Women's Se...|
|B012VOPU84|iLoveSIA Women's ...|
|B012UUEWZG|Harley-Davidson W...|
|B012LQU4CE|Women's Laced & L...|
|B012ERJSKO|Amoin Style Women...|
|B012ADLW92|Cocoship Retro 50...|
|B012980QUY|Sanwood® Cute Bab...|
|B0125V8SZK|OURS Women's Comf...|
|B0125M6TZ0|Modern Kiwi Solid...|
|B01254G39K|Alberto Cardinali...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|review_date|
+--------------+-----------+----------+--------------+-----------+-----------+
|R1KKOXHNI8MSXU|   32158956|B01KL6O72Y|      24485154| 2013-01-14| 2013-01-14|
|R26SP2OPDK4HT7|    2714559|B01ID3ZS5W|     363128556| 2014-03-04| 2014-03-04|
| RWQEDYAX373I1|   12608825|B01I497BGY|     811958549| 2015-07-12| 2015-07-12|
|R231YI7R4GPF6J|   25482800|B01HDXFZK6|     692205728| 2015-06-03| 2015-06-03|
|R3KO3W45DD0L1K|    9310286|B01G6MBEBY|     431150422| 2015-06-12| 2015-06-12|
+--------------+-----------+----------+--------------+-----------+-----------+
only showing top 5 rows



In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1KKOXHNI8MSXU|          4|            0|          0|   N|                Y|
|R26SP2OPDK4HT7|          5|            1|          2|   N|                Y|
| RWQEDYAX373I1|          5|            0|          0|   N|                Y|
|R231YI7R4GPF6J|          5|            0|          0|   N|                Y|
|R3KO3W45DD0L1K|          5|            0|          0|   N|                Y|
|R1C4QH63NFL5NJ|          5|            0|          0|   N|                Y|
|R2GP65O1U9N7BP|          5|            0|          0|   N|                Y|
|R3O29CT5MQQ3XQ|          4|            0|          0|   N|                Y|
|R1ZECD2AA8QFF6|          5|            0|          0|   N|                Y|
|R2S79GCF6J89OA|          3|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://module16.cxzh1xggreey.us-east-2.rds.amazonaws.com:5432/module16"
config = {"user":"postgres", 
          "password": "881125zy", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)